In [1]:
import numpy as np
from scipy.stats import multivariate_normal as norm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Loading the training and test data

## Train data

In [2]:
train5 = np.loadtxt("mnist_all/train5.txt")/255 # /255 for normalization
train6 = np.loadtxt("mnist_all/train6.txt")/255
train8 = np.loadtxt("mnist_all/train8.txt")/255

In [3]:
# Define targets 
train5_target = 5*np.ones(len(train5))
train6_target = 6*np.ones(len(train6))
train8_target = 8*np.ones(len(train8))

In [4]:
# Combine data
train_data = np.concatenate([train5, train6, train8])
train_targets = np.concatenate([train5_target, train6_target, train8_target])

## Test data

In [5]:
test5 = np.loadtxt("mnist_all/test5.txt")/255
test6 = np.loadtxt("mnist_all/test6.txt")/255
test8 = np.loadtxt("mnist_all/test8.txt")/255

# Define targets 
test5_target = 5*np.ones(len(test5))
test6_target = 6*np.ones(len(test6))
test8_target = 8*np.ones(len(test8))

# Combine
test_data = np.concatenate([test5, test6, test8])
test_targets = np.concatenate([test5_target, test6_target, test8_target])

# Class names
classes = np.array([5,6,8])

# Part 1: Reduce dimension to 2
Here, we wish to reduce the data dimensionality from 784 to 2 using either PCA or LDA.
For this you can use scikit-learn.

## PCA

The PCA class in scikit-learn fits a covariance matrix and compute eigenvectors for you. 
PCA doesn't assume any knowledge about the classes, so you have to use the concatenated training set.

In [6]:
# Fit a scikit learn PCA instance to training data

Now that the PCA model is fit to the training data, we can find a low dimesional representation of each class.

In [7]:
# Transform train data from each class using fitted PCA instance

## LDA
We can also use Linear Disicriminant Analysis to reduce the dimensionality of the data.
The LDA class in scikit-learn fits a covariance matrix and compute eigenvectors for you. LDA assume that you know about the classes, so you have to use the concatenated training set and targets/classes

In [8]:
# Fit a scikit learn LDA instance to training data

In [9]:
# Transform train data from each class using fitted LDA instance

Let's try to plot the dimensionality reduced data and compare PCA to LDA. What do we see?

In [10]:
#Scatter plot of the dimensional-reduced data 


In the above plot we see that LDA is seemingly better at seperating the tree classes,while the classes 5 and 8 are highly overlapped when using PCA.

# Part 2: Perform 3-class classification based on the generated 2-dimensional data. 
We need to find a model to classify the test data as either 5, 6, or 8.
Here, we could use a Gaussian model for each class, and estimate the mean and covariance from the dimensionality reduced data.

## Estimate Gaussians using 2-dimensional data obtained from PCA

In [11]:
#Estimate parameters for a bivariante Gaussian distribution.


## Estimate Gaussians using 2-dimensional data obtained from LDA

In [12]:
#Estimate parameters for a bivariante Gaussian distribution.


## Classifying test data
To classify the test data, we first transform it to 2-dimensions as well.

In [13]:
# Transform test data using fitted PCA/LDA instance

Now we compute priors, likelihoods and posteriors

In [14]:
# Compute priors
# Compute Likelihoods
# Compute posteriors

We can now compute the classification accuracy for both PCA and LDA

In [15]:
#Compute predictions

#Compute accuracy


What does the results show?